### Library installation and script setup
This box needs to only be run once. It builds the environment to carry out the rest of the analysis.

In [2]:
# Run one time only - install pip and unusual Libraries
import pip
import pandas as pd
import shapely
from shapely.wkt import loads
from shapely.geometry import box
from shapely.geometry import MultiPolygon
import time
import json
from gbdxtools import Interface
from gbdxtools.task import env
from gbdxtools import CatalogImage
gbdx = Interface()
%matplotlib inline
import geopandas as gpd

### Define AOI and bounding box of AOI from shapefile
The user needs to have their Area of Interest ("AOI") defined as a one-polygon shapefile. This should be uploaded to the Jupyter server. Upload a polygon shapefile with only one feature. Save this file to the Jupyter server (click on the Jupyter logo, then click 'upload'). This step should be performed before running the next cell. 

In [3]:
# determine if input is single point or shapefile
input_type = 'shapefile'

if input_type == 'point':
    from shapely.geometry import Point
    
    # Create shapely Point instance 
    point = Point((92.1443285, 21.2705219))
    
    # Make a square box around it using buffer method
    point = point.buffer(0.002, cap_style = 3)
    AOI = point
    
elif input_type == 'shapefile': 
    # Attach the name of uploaded file to the fname variable. If it is not 'shapefile.shp', change the name accordingly. 
    fname = 'Padma_1.shp' 

    # Create Area of Interest (AOI) object as a geopandas object; reproject to WGS84 if not already in WGS 84
    AOI = gpd.read_file(fname)
    crs_WGS84 = {'init' :'epsg:4326'}
    AOI = AOI.to_crs(crs_WGS84)

    # Redefine AOI as the first WKT object in the DataFrame (valid for 1-polygon objects only)
    AOI = AOI['geometry'].loc[0]

else: 
    print('** WARNING: Failure to determine adequate input data type **')
    AOI = ''
    
# Create bboxx - the square shaped box which will always contain the AOI.
bboxx = []
for coord in range(0,len(AOI.bounds)): 
    bboxx.append(AOI.bounds[coord])

# Print to screen - AOI and bboxx
print 'AOI: || ', AOI, '\n\nbboxx: || ', bboxx

AOI: ||  POLYGON ((90.13814810386185 23.38893329021568, 90.3320378864282 23.75250366283122, 90.4669528601306 23.70591047355641, 90.29164418172688 23.32069846171082, 90.13814810386185 23.38893329021568)) 

bboxx: ||  [90.13814810386185, 23.320698461710816, 90.4669528601306, 23.75250366283122]


### Perform Initial Vector Search: AOI Vector Against Image Catalog for All Potential Images
Given the calculations conducted in the previous step, this cell queries the DigitalGlobe image database based on the vector footprint of the AOI defined above, and returns a list of catalog IDs which at least partially cover the AOI. As default, it returns up to 1000 images, with a cloud cover less than 25%. 

In [4]:
# Define search function. Returns up to 1000 images where cloud cover smaller than 25%
def search_unordered(bbox, _type, count=1000, cloud_cover=25):
    aoi = AOI.wkt
    query = "item_type:{} AND item_type:DigitalGlobeAcquisition".format(_type)
    query += " AND attributes.cloudCover_int:<{}".format(cloud_cover)
    return gbdx.vectors.query(aoi, query, count=count)

# Run search on Area of Interest (AOI). Passes in AOI in Well Known Text format (wkt)
records = search_unordered(AOI.wkt, 'DigitalGlobeAcquisition')

# Create list object of all catalog IDs returned in search
ids = [r['properties']['attributes']['catalogID'] for r in records]

# Print length of list of catalogi IDs (number of unique images)
print 'Unique images found: %s' % len(ids)

Unique images found: 253


### Generate CSV of images that match criteria
At this point, the script generates a .csv file with the key metadata about the image which will be used for further processing later. This is the definitive database for the given AOI. No filtering yet takes place.

Key calculated statistics include: the area where the scene and the AOI don't overlap (AA), the corresponding overlap with the image (BB), the fraction of the AOI covered (frac), and the binary flag for whether the image is yet in IDAHO. 

In [5]:
%matplotlib inline

# Define Counters
l = 0    # number of non-IDAHO images
scenes = [] # list containing metadata dictionaries of all scenes in our AOI 

# Toggle for printing images to screen
download_thumbnails = 0

# Loop catalog IDs
for i in ids:
    
    # Fetch metadata dictionary for each catalog ID in ids list
    r = gbdx.catalog.get(i)
    
    # Check location of ID - is it in IDAHO?
    location = gbdx.catalog.get_data_location(i)
    
    # Defines IDAHO variable as binary 1 / 0 depending on whether it is in IDAHO already or not
    if location == 'not_delivered':
        l = l + 1
        idaho = 0
    else:
        idaho = 1
    
        # Download image if image in IDAHO and toggle on
        if download_thumbnails == 1:
            image = CatalogImage(i, band_type="MS", bbox=bboxx)
            image.plot(w=10, h=10)
        else:
            pass
    
    # Print statement to consol for key image variables
    print 'ID: %s, Timestamp: %s, Cloud Cover: %s, Image bands: %s, IDAHO: %s' % (i,r['properties']['timestamp'],r['properties']['cloudCover'],r['properties']['imageBands'],idaho)
    
    # Calculate the percentage overlap with our AOI for each scene
    # load as a Shapely object the wkt representation of the scene footprint
    footprint = r['properties']['footprintWkt']
    shapely_footprint = shapely.wkt.loads(footprint)
    
    # Calculate the object that represents the difference between the AOI and the scene footprint 
    AA = AOI.difference(shapely_footprint)
    
    # Define frac as the fraction, between 0 and 1, of the AOI that the scene covers
    frac = 1 - ((AA).area / AOI.area)
    
    # Create BB - the proxy for the useful area. IF scene entirely contains AOI, then BB = AOI, else it is the intersection 
    # of the scene footprint and the AOI
    BB = AOI 
    if frac < 1:
        BB = AOI - AA
    #shapely_footprint.intersection(AOI)
    # Similarly, AA, the difference area between AOI and the scene, can be set to null if the scene contains 100% of the AOI 
    if frac == 1:
        AA = ""
        
    # Append key metadata to list obejct 'scenes' for the current scene, as a dictionary. This then moves into the pandas dataframe.
    # Several objects here are from DigitalGlobe's metadata dictionary (anything with an r start)
    scenes.append({
        'ID':i, 
        'TimeStamp':r['properties']['timestamp'],
        'CloudCover':r['properties']['cloudCover'],
        'ImageBands':r['properties']['imageBands'],
        'On_IDAHO':idaho,
        'browseURL': r['properties']['browseURL'],
        'Overlap_%': frac * 100,
        'PanResolution': r['properties']['panResolution'],
        'MultiResolution': r['properties']['multiResolution'],
        'OffNadirAngle': r['properties']['offNadirAngle'],
        'Sensor':r['properties']['sensorPlatformName'],
        'Full_scene_WKT':r['properties']['footprintWkt'],
        'missing_area_WKT':AA,
        'useful_area_WKT':BB
        })
    
# Summary Statistics - show totals for images, both in IDAHO and currenlty unavailable images. 
print 'Number of catalog IDs not available in IDAHO: %s' % l
print 'Number of catalog IDs available in IDAHO: %s' % (len(ids) - l)

# Define column order for dataframe of search results
cols = ['ID','Sensor','ImageBands','TimeStamp','CloudCover','Overlap_%','PanResolution','MultiResolution','OffNadirAngle','On_IDAHO','browseURL','Full_scene_WKT','useful_area_WKT','missing_area_WKT']

#Generate pandas dataframe from results
out = pd.DataFrame(scenes,columns = cols)

ID: 103001005F603400, Timestamp: 2016-11-01T04:42:55.314Z, Cloud Cover: 11, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 1040010013229A00, Timestamp: 2015-11-07T04:57:15.397Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 103001005F73F900, Timestamp: 2016-11-01T04:41:30.164Z, Cloud Cover: 9, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 1020010009B7BE00, Timestamp: 2009-09-29T04:51:58.816Z, Cloud Cover: 19, Image bands: PAN, IDAHO: 0
ID: 1020010009C9D500, Timestamp: 2009-09-29T04:50:55.416Z, Cloud Cover: 21, Image bands: PAN, IDAHO: 0
ID: 1020010009CB7900, Timestamp: 2009-10-12T04:45:38.980Z, Cloud Cover: 5, Image bands: PAN, IDAHO: 0
ID: 103001000386FD00, Timestamp: 2009-12-06T04:41:55.949Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 10400100250FF800, Timestamp: 2016-12-30T04:53:49.612Z, Cloud Cover: 2, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 101001000362D500, Timestamp: 2004-11-13T04:46:46.663Z, Cloud Cover: 0, Image bands: PAN_MS1, IDAHO: 0
ID: 1010010005624500, Timestamp: 20

ID: 102001000929F300, Timestamp: 2009-09-29T04:51:08.016Z, Cloud Cover: 21, Image bands: PAN, IDAHO: 0
ID: 102001000BE1DE00, Timestamp: 2010-01-30T05:01:51.385Z, Cloud Cover: 0, Image bands: PAN, IDAHO: 0
ID: 1010010008DDA500, Timestamp: 2008-11-29T04:54:11.438Z, Cloud Cover: 0, Image bands: PAN_MS1, IDAHO: 0
ID: 1050410004B18900, Timestamp: 2013-12-22T04:42:13.684Z, Cloud Cover: 7, Image bands: PAN_MS1, IDAHO: 0
ID: 1010010009114B00, Timestamp: 2009-01-17T04:48:34.642Z, Cloud Cover: 0, Image bands: PAN_MS1, IDAHO: 0
ID: 10500100078B6400, Timestamp: 2016-12-21T04:39:02.685Z, Cloud Cover: 0, Image bands: PAN_MS1, IDAHO: 0
ID: 103001001F9E0000, Timestamp: 2013-02-22T04:50:37.790Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 1050410001313700, Timestamp: 2010-12-26T04:36:38.485Z, Cloud Cover: 0, Image bands: PAN_MS1, IDAHO: 0
ID: 101001000AE71E00, Timestamp: 2010-01-07T04:37:07.846Z, Cloud Cover: 23, Image bands: PAN_MS1, IDAHO: 0
ID: 102001002DC4ED00, Timestamp: 2014-03-20T04:4

ID: 103001002A86C200, Timestamp: 2013-12-20T05:05:05.440Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 1050410004BE5400, Timestamp: 2014-01-26T04:33:29.484Z, Cloud Cover: 0, Image bands: PAN_MS1, IDAHO: 0
ID: 102001000F655500, Timestamp: 2010-10-04T04:58:25.021Z, Cloud Cover: 12, Image bands: PAN, IDAHO: 0
ID: 105041000F7ACB00, Timestamp: 2014-01-26T04:34:20.485Z, Cloud Cover: 0, Image bands: PAN_MS1, IDAHO: 0
ID: 10504100107DCD00, Timestamp: 2014-05-12T04:36:44.485Z, Cloud Cover: 0, Image bands: PAN_MS1, IDAHO: 0
ID: 1040010029BF9100, Timestamp: 2017-02-25T05:11:53.353Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 103001005D88E800, Timestamp: 2016-11-09T04:46:58.576Z, Cloud Cover: 9, Image bands: PAN_MS1_MS2, IDAHO: 1
ID: 105041000F4D7E00, Timestamp: 2013-12-11T04:36:16.484Z, Cloud Cover: 0, Image bands: PAN_MS1, IDAHO: 0
ID: 10504100101F3000, Timestamp: 2014-03-05T04:35:12.485Z, Cloud Cover: 0, Image bands: PAN_MS1, IDAHO: 0
ID: 1010010012782000, Timestamp: 2014

ID: 1030010085896800, Timestamp: 2018-10-24T04:54:23.000Z, Cloud Cover: 4, Image bands: PAN_MS1_MS2, IDAHO: 1
ID: 1030010086BC5500, Timestamp: 2018-10-24T04:55:47.000Z, Cloud Cover: 3, Image bands: PAN_MS1_MS2, IDAHO: 1
ID: 10300100181FF500, Timestamp: 2012-05-10T05:05:41.183Z, Cloud Cover: 18, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 102001001E417D00, Timestamp: 2012-10-22T04:42:54.387Z, Cloud Cover: 1, Image bands: PAN, IDAHO: 0
ID: 1040010037380300, Timestamp: 2018-01-21T05:10:14.000Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 1040010038438F00, Timestamp: 2018-01-21T05:09:42.000Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 0
ID: 10400100378C8600, Timestamp: 2018-01-21T05:09:57.000Z, Cloud Cover: 0, Image bands: PAN_MS1_MS2, IDAHO: 1
ID: 10504100005EE300, Timestamp: 2012-02-22T04:47:27.484Z, Cloud Cover: 0, Image bands: PAN_MS1, IDAHO: 0
ID: 13ff9a0f-e846-4ab3-a664-99cc33f298d9-inv, Timestamp: 2017-11-27T04:37:17.000Z, Cloud Cover: 0, Image bands: PAN_MS1, IDAHO: 0
I

### Perform Categorical Search: Remove Disqualified Images
Performs pandas .loc operations to cut out scenes which don't meet the criteria specified by the 'cutoff' series of variables. Define these in-script in the '# Define categorical search parameters' section. 

In [7]:
# Convert Timestamp field to pandas DateTime object
out['TS'] = out['TimeStamp'].apply(lambda x: pd.Timestamp(x))

# Add separate date and time columns for easy interpretation
string = out['TimeStamp'].str.split('T')
out['Date'] = string.str.get(0)
out['Time'] = string.str.get(1)

# Define categorical search parameters
cutoff_cloud_cover = 2   # images with CC over this threshold discarded
cutoff_overlap = 0     # images with AOI overlap below this threshold discarded. [N.b.: keep small if AOI large.]
cutoff_date_upper = '1-Jan-19'  # images newer than this date discarded
cutoff_date_lower = '1-Jan-07'  # images older than this date discarded\
cutoff_nadir = 30 # Images at nadir angles greater than threshold discarded
cutoff_pan_res = 1 # Images below this resolution discarded
accepted_bands = ['PAN_MS1','PAN_MS1_MS2'] #  Images with any other band entry discarded

# Categorical Search: remove disqualified images. Copy of dataframe taken, renamed to 'out_1stcut'.
out_1stcut = out.loc[(out['CloudCover'] <= cutoff_cloud_cover) & 
                     (out['Overlap_%'] >= cutoff_overlap) & 
                     (out['TS'] > pd.Timestamp(cutoff_date_lower, tz = 0)) & 
                     (out['TS'] < pd.Timestamp(cutoff_date_upper, tz = 0)) &
                     (out['ImageBands'].isin(accepted_bands)) & 
                     (out['OffNadirAngle'] <= cutoff_nadir) & 
                     (out['PanResolution'] <= cutoff_pan_res)
                    ]

# Print to consol remaining images after categorical search undertaken
print '\nNumber of results remaining: %s' % len(out_1stcut.index)


Number of results remaining: 131


### Apply Image Quality Ranking
For the remaining images that met minimum quality thresholds, rank them, best to worst. This is done on a points-based system. Images accrue points for: 
- every % of cloud cover (1 point)
- every % of missed overlap with the AOI (1 point)
- every week away from the optimal date (1 point)
- every degree away from nadir (1 point)
- every cm of resolution worse than the optimal resolution 

User preferences are defined in the 'pref_weights' dictionary. 
Ensure all weights sum to 1.

In [8]:
# Define continuous image ranking preferences
optimal_date =  '1-Jan-19' # Optimal date (enter as dd-mmm-yy)
optimal_pan_res = 0.4 # Optimal pan resolution, metres
optimal_nadir = 0 # optimal image angle. 0 = vertical

# Define continuous image ranking preference weights. Must sum to 1.
# If user cares more about scenes being contemporaneous, up 'date' weighting at expense of other categories. 
pref_weights = {
    'cloud_cover': 0.5,
    'overlap':0.5,
    'date': 0.0,
    'nadir': 0.0,
    'resolution': 0.0
    }

# Apply ranking method over all non-disqualified search results for each field
optimal_date = pd.to_datetime(optimal_date, utc = True)

# each 1% of cloud cover = 1 point
out_1stcut['points_CC'] = (out_1stcut['CloudCover'])  

# each 1% of overlap missed = 1 point
out_1stcut['points_Overlap'] = (100 - out_1stcut['Overlap_%'])  

# each week away from the optimal date = 1 point 
out_1stcut['points_Date'] = ((abs(out_1stcut['TS'] - optimal_date)).view('int64') / 60 / 60 / 24 / 1E9) / 7 

# each degree off nadir = 1 point
out_1stcut['points_Nadir'] = abs(out_1stcut['OffNadirAngle'] - optimal_nadir) 

# each cm of resolution worse than the optimal resolution = 1 point
out_1stcut['points_Res'] = (out_1stcut['PanResolution'] - optimal_pan_res).apply(lambda x: max(x,0)) * 100 

# Define ranking algorithm - weight point components defined above by the preference weighting dictionary
def Ranker(out_1stcut, pref_weights):
    a = out_1stcut['points_CC'] * pref_weights['cloud_cover']
    b = out_1stcut['points_Overlap'] * pref_weights['overlap']
    c = out_1stcut['points_Date'] * pref_weights['date'] 
    d = out_1stcut['points_Nadir'] * pref_weights['nadir']
    e = out_1stcut['points_Res'] * pref_weights['resolution']
    
    # Score is linear addition of the number of 'points' the scene wins as defined above. More points = worse fit to criteria
    rank = a + b + c + d + e
    return rank

# Add new column - Rank Result - with the total number of points accrued by the scene 
out_1stcut['RankResult'] = Ranker(out_1stcut,pref_weights)

# Add a Preference order column - Pref_Order - based on Rank Result, sorted ascending (best scene first)
out_1stcut = out_1stcut.sort_values(by = 'RankResult', axis = 0, ascending = True)
out_1stcut = out_1stcut.reset_index()
out_1stcut['Pref_order'] = out_1stcut.index + 1
out_1stcut = out_1stcut.drop(['index'], axis = 1)

### Output image results .csv
Before construction of the scene mosaic, output a .csv with the details of selected and ranked scenes, named 'Scene_List.csv'


In [41]:
cols = ['ID','Sensor','ImageBands','Date','Time','CloudCover','Overlap_%','PanResolution','MultiResolution','OffNadirAngle','On_IDAHO','Pref_order','RankResult','points_CC','points_Overlap','points_Date','points_Nadir','points_Res','browseURL','Full_scene_WKT','useful_area_WKT','missing_area_WKT']
out_1stcut = out_1stcut[cols]
out_1stcut['AOI_WKT'] = AOI
out_1stcut.to_csv(os.path.join('C:\Users\charl\Documents\GOST\GBDX\Padma','Scene_List.csv'))

### Build composite shape from useful area WKT

This block takes the AOI and subtracts from it, in rank-preferred order, the useable scene area for each catalog ID. This results in a diminishing 'AOI remaining' area, which the loop seeks to fill, with ever decreasing 'utility' images. A new column in the dataframe is created - 'used_area_wkt' - which is the geometry of the image which should be requested if trying to make a mosaic of these catalog IDs. 

In [25]:
# Create a new copy of the dataframe to work on
finaldf = out_1stcut

# Add column for used scene region area, expressed as .wkt
finaldf['used_scene_region_WKT'] = 0
finaldf['used_area'] = 0

# Set initial value of AOI_remaining to the full AOI under consideration
AOI_remaining = AOI

# Create two lists - usedareas for the areas of scenes used in the final product, and AOI_rems to record sequential reduction in 
# remaining AOI that needs to be filled
usedareas = []
AOI_rems = []

# Set up loop for each image in dataframe of ranked images
for s in finaldf.index:
    print 'calculating for %s' % s
    
    # pick up the WKT of the useful area as the useful_scene_region variable
    useful_scene_region = finaldf['useful_area_WKT'].loc[s]
    
    # Set up try loop - to catch if there is no intersection of AOI_remaining and useful_scene_region
    try: 
        
        # define 'used_scene_region' as the useable bit of the image that overlaps the AOI
        used_scene_region = AOI_remaining.intersection(useful_scene_region)
        
        # calculate the area of that region
        used_area = used_scene_region.area
        
        # Check to see if this is a geometry collection. This shapely type if for 'jumbles' of outputs (e.g. Polygons + Lines)
        # This can be created if the intersection process decides that it also wants a 1-pixel strip from the bottom of the image
        # as well as the main chunk. This won't translate back to a shapefile, so we drop non-Polygon objects iteratively. 
        if used_scene_region.type == 'GeometryCollection':
            xlist = []
            
            # Iterate through all objects in the geometry collection
            for y in used_scene_region.geoms:
                
                # Add polygons to a fresh list
                if y.type == 'Polygon':
                    xlist.append(y)
                    
            # Convert that list to a multipolygon object
            used_scene_region = MultiPolygon(xlist)
        else:
            pass
        
        # Append the used bit of the image to the usedareas list. 
        usedareas.append(used_scene_region)
        
        # Add two new columns to the dataframe - the used scene geometry in wkt, and the area of the used scene
        finaldf['used_scene_region_WKT'].loc[s] = used_scene_region
        finaldf['used_area'].loc[s] = used_area
        
        # Redefine the area of the AOI that needs to be filled by the next, lower-rank image
        AOI_remaining = AOI_remaining.difference(used_scene_region)
        
        # Add this to the AOI_rems list for troubelshooting and verification
        AOI_rems.append(AOI_remaining)
    except:
        pass

calculating for 0
calculating for 1
calculating for 2
calculating for 3
calculating for 4
calculating for 5
calculating for 6


### Output and Summary
End of the process - print how much of the AOI was eventually covered with these settings, and send the .csv to file as 'Final_Scene_List'

In [26]:
# Drop from the scene list any scene where the area used is less than 1% of the AOI
finaldf = finaldf.loc[finaldf['used_area'] > (AOI.area / 100)]

# Print summary statistics to consol
print 'Remaining images: %s' % len(finaldf.index)
print 'Proportion of AOI covered: %d percent' % (finaldf['used_area'].sum() / AOI.area * 100)
print 'Remaining area:'
AOI_remaining

# Send final .csv to file
finaldf.to_csv('Final_Scene_List_Konna.csv')

Remaining images: 2
Proportion of AOI covered: 100 percent
Remaining area:


In [42]:
o = out_1stcut.copy()
o = o.sort_values(by = 'Overlap_%', ascending = False)[:20]

In [30]:
order_list = list(o.ID)

order_receipts = []
print 'Number of images to be ordered: %d' % len(order_list)

consent = 'I agree to ordering these image IDs to IDAHO'

if consent == 'I agree to ordering these image IDs to IDAHO':
    for x in order_list:
        try:
            order_id = gbdx.ordering.order(x)
            order_receipts.append(order_id)
        except:
            order_receipts.append('error!')
else: 
    print 'please write out your consent in the consent variable above'

Number of images to be ordered: 20


In [32]:
for receipt in order_receipts[:20]:
    try:
        print(gbdx.ordering.status(receipt))
    except:
        print('error!')

[{u'acquisition_id': u'103001001F9E0000', u'state': u'delivered', u'location': u's3://receiving-dgcs-tdgplatform-com/059092634010_01_003'}]
[{u'acquisition_id': u'1030010064CCE300', u'state': u'delivered', u'location': u's3://receiving-dgcs-tdgplatform-com/058881809010_01_003'}]
[{u'acquisition_id': u'1030010077677700', u'state': u'delivered', u'location': u's3://receiving-dgcs-tdgplatform-com/058385670010_01_003'}]
error!
[{u'acquisition_id': u'1030010063CEDE00', u'state': u'delivered', u'location': u's3://receiving-dgcs-tdgplatform-com/059092635010_01_003'}]
error!
[{u'acquisition_id': u'1030010003D0A300', u'state': u'delivered', u'location': u's3://receiving-dgcs-tdgplatform-com/059092636010_01_003'}]
[{u'acquisition_id': u'1050410004BE5400', u'state': u'delivered', u'location': u's3://receiving-dgcs-tdgplatform-com/059092637010_01_003'}]
[{u'acquisition_id': u'105041000F7ACA00', u'state': u'delivered', u'location': u's3://receiving-dgcs-tdgplatform-com/059092638010_01_003'}]
[{u'ac

In [37]:
import sys, os
sys.path.insert(0, r"C:\Users\charl\Documents\GitHub\GOST_GBDX")
from GOST_GBDx_Tools import gbdxTasks
from GOST_GBDx_Tools import gbdxURL_misc
from gbdxtools import CatalogImage

curTasks = gbdxTasks.GOSTTasks(gbdx)
gbdxUrl = gbdxURL_misc.gbdxURL(gbdx)

curFolder=r'C:\Users\charl\Documents\GOST\GBDX\Padma'

In [47]:
for root, dirs, files in os.walk(curFolder, topdown=False):
    pass

i = 0
output = o
for i in range(0, len(output)):
    
        cWKT = output.AOI_WKT.iloc[i]
        catID = output.ID.iloc[i]
        curFile = 'img_%s.tif' % (catID)
         
        a = os.path.join(curFolder, curFile)

        if curFile in files:
            print('file already exists, skipping download: %s' % curFile)
            pass
        else:
            print('downloading: %s' % curFile)
            curTasks.downloadImage(catID, a, boundingBox=cWKT.bounds, imgChipSize = 100000)

downloading: img_103001001F9E0000.tif


CPLE_FileIO: Free disk space available is 16373051392 bytes, whereas 180824557440 are at least necessary. You can disable this check by defining the CHECK_DISK_FREE_SPACE configuration option to FALSE.

In [58]:
t = list(o.ID.sort_values(ascending = False))[3:]

In [60]:
print(t)

[u'1050410012640600', u'105041000F7ACB00', u'105041000F7ACA00', u'1050410004BE5400', u'1050410004BC4000', u'1050410001EC1E00', u'1050410001313700', u'10504100005EE300', u'10500100078B6600', u'1040010045AEE400', u'1030010077677700', u'1030010064CCE300', u'1030010063CEDE00', u'1030010054688100', u'103001001F9E0000', u'1030010011388F00', u'1030010003D0A300']


In [ ]:
k = '1050410012640600,
105041000F7ACB00, 105041000F7ACA00, 1050410004BE5400, 1050410004BC4000, 1050410001EC1E00, 1050410001313700, 10504100005EE300, 10500100078B6600, 1040010045AEE400, 1030010077677700, 1030010064CCE300, 1030010063CEDE00, 1030010054688100, 103001001F9E0000, 1030010011388F00, 1030010003D0A300'